In [ ]:
import numpy as np
from numpy import load
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano' # Using theano as backend instead of tensorflow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

plt.switch_backend('agg')
%matplotlib inline

Using Theano backend.


In [ ]:
# Mounting google drive, depends wherever the data is
from google.colab import drive 
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# path to dataset
path_to_dataset = 'drive/My Drive/dataset.csv'
# path to transformer generated chunk embeddings eg. XLNet etc.
path_to_transformer_chunk_embeddings_train = 'drive/My Drive/XLNet/XLNet_train.npy'
path_to_transformer_chunk_embeddings_dev = 'drive/My Drive/XLNet/XLNet_dev.npy'
path_to_transformer_chunk_embeddings_test = 'drive/My Drive/XLNet/XLNet_test.npy'

In [ ]:
# loading dataset
dataset = pd.read_csv(path_to_dataset)

In [ ]:
# loading the chunks embeddings
x_train0 = load(path_to_transformer_chunk_embeddings_train, allow_pickle = True)
x_dev0 = load(path_to_transformer_chunk_embeddings_dev, allow_pickle= True)
x_test0 = load(path_to_transformer_chunk_embeddings_test, allow_pickle= True)

In [ ]:
# loading the corresponding label for each case in dataset
dev = dataset.loc[dataset['split'] == 'dev']
train = dataset.loc[dataset['split'] == 'train']
test = dataset.loc[dataset['split'] == 'test']

y_train0 = []
for i in range(train.shape[0]):
  y_train0.append(train.loc[i,'label'])

y_dev0 = []
for i in range(dev.shape[0]):
  y_dev0.append(dev.loc[i+32305,'label'])

y_test0 = []
for i in range(test.shape[0]):
  y_test0.append(test.loc[i+33299,'label'])

In [ ]:
# Setting the maximum sequnce length and embedding dimension
MAX_SEQUENCE_LENGTH = 25
EMBEDDING_DIM = 768

In [ ]:
# padding the vectors to maximum sequence length
for i in range(x_train0.shape[0]):
  padding_vector = np.zeros((MAX_SEQUENCE_LENGTH - x_train0[i].shape[0], EMBEDDING_DIM))
  x_train0[i] = np.concatenate((x_train0[i],padding_vector), axis = 0)
    
for i in range(x_dev0.shape[0]):
  padding_vector = np.zeros((MAX_SEQUENCE_LENGTH - x_dev0[i].shape[0], EMBEDDING_DIM))
  x_dev0[i] = np.concatenate((x_dev0[i],padding_vector),axis = 0)
    
for i in range(x_test0.shape[0]):
  padding_vector = np.zeros((MAX_SEQUENCE_LENGTH - x_test0[i].shape[0], EMBEDDING_DIM))
  x_test0[i] = np.concatenate((x_test0[i],padding_vector), axis = 0)

In [ ]:
# Using Input layer to convert into required tensor shape
text_input = Input(shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), dtype='float32', name='text')
# Using 3 Conv1D layers followed by max pooling layers
l_cov1= Conv1D(256, 2, activation='relu')(text_input)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 = Conv1D(128, 2, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(2)(l_cov2)
l_cov3 = Conv1D(128, 2, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(4)(l_cov3)  # global max pooling
# Using the flatten layer to convert into 1D tensor
l_flat = Flatten()(l_pool3)
# passing the output embeddings through 2 dense layers
l_dense = Dense(128, activation='relu')(l_flat)
l_dense1 = Dense(32, activation='relu')(l_dense)
# Using sigmoid classifier
preds = Dense(1, activation='sigmoid')(l_dense1)

model = Model(text_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()

Simplified convolutional neural network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            (None, 25, 768)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 256)           393472    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11, 128)           65664     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 128)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 4, 128)            32896     
_________________________________________________________________
max_pooling1d_3 (Ma

In [ ]:
num_sequences = len(x_train0)
batch_size = 32
batches_per_epoch =  int(num_sequences/batch_size)
num_features= 768
def train_generator(): # function to generate batches of corresponding batch size
    x_list= x_train0
    y_list =  y_train0
    # Generate batches
    while True:
        for b in range(batches_per_epoch):
            longest_index = (b + 1) * batch_size - 1
            timesteps = len(max(x_train0[:(b + 1) * batch_size][-batch_size:], key=len))
            x_train = np.full((batch_size, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size,  1))
            for i in range(batch_size):
                li = b * batch_size + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
num_sequences_val = len(x_dev0)
batch_size_val = 32
batches_per_epoch_val = int(num_sequences_val/batch_size_val)
num_features= 768
def val_generator(): # function to generate batches of corresponding batch size
    x_list= x_dev0
    y_list =  y_dev0
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(x_dev0[:(b + 1) * batch_size_val][-batch_size_val:], key=len))
            x_train = np.full((batch_size_val, timesteps, num_features), 0)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
num_features= 768
def test_generator(): # function to generate batches of corresponding batch size
    x_list= x_test0
    y_list =  y_test0
    # Generate batches
    while True:
        for b in range(batches_per_epoch_test):
            if(b == batches_per_epoch_test-1): # An extra if else statement just to manage the last batch as it's size might not be equal to batch size 
              longest_index = num_sequences_test - 1
              timesteps = len(max(x_test0[:longest_index + 1][-batch_size_test:], key=len))
              x_train = np.full((longest_index - b*batch_size_test, timesteps, num_features), -99.)
              y_train = np.zeros((longest_index - b*batch_size_test,  1))
              for i in range(longest_index - b*batch_size_test):
                  li = b * batch_size_test + i
                  x_train[i, 0:len(x_list[li]), :] = x_list[li]
                  y_train[i] = y_list[li]
            else:
                longest_index = (b + 1) * batch_size_test - 1
                timesteps = len(max(x_test0[:(b + 1) * batch_size_test][-batch_size_test:], key=len))
                x_train = np.full((batch_size_test, timesteps, num_features), -99.)
                y_train = np.zeros((batch_size_test,  1))
                for i in range(batch_size_test):
                    li = b * batch_size_test + i
                    x_train[i, 0:len(x_list[li]), :] = x_list[li]
                    y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
# Setting the callback and training the model
call_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=2, verbose=2,
                                mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

model.fit_generator(train_generator(), steps_per_epoch=batches_per_epoch, epochs=3,
                    validation_data=val_generator(), validation_steps=batches_per_epoch_val, callbacks =[call_reduce] )

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Epoch 1/3
1009/1009 [==============================] - 130s 129ms/step - loss: 0.4970 - acc: 0.7544 - val_loss: 0.4926 - val_acc: 0.7772
Epoch 2/3
1009/1009 [==============================] - 137s 136ms/step - loss: 0.4724 - acc: 0.7691 - val_loss: 0.5014 - val_acc: 0.7762
Epoch 3/3
1009/1009 [==============================] - 142s 141ms/step - loss: 0.4584 - acc: 0.7793 - val_loss: 0.4891 - val_acc: 0.7853

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.


In [ ]:
num_sequences_test = len(x_test0)
batch_size_test = 32
batches_per_epoch_test = int(num_sequences_test/batch_size_test) + 1
num_features= 768
# evaluating on the test data
model.evaluate_generator(test_generator(), steps= batches_per_epoch_test)

[0.548132598400116, 0.7691292762756348]

In [ ]:
# defining a function which calculates various metrics such as micro and macro precision, accuracy and f1
def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1

In [ ]:
# getting the predicted labels on the test data
preds = model.predict_generator(test_generator(), steps= batches_per_epoch_test)
y_pred = preds > 0.5

# Calculating all metrics on test data predicted label
print(metrics_calculator(y_pred, y_test0[:-1]))

(0.7789586641591649, 0.7687549494826431, 0.7738231714660547, 0.7691292875989446, 0.7691292875989446, 0.7691292875989446)


In [ ]:
# getting the predicted labels on the dev data
preds = model.predict_generator(val_generator(), steps= batches_per_epoch_val)
y_pred_dev = preds > 0.5

# Calculating all metrics on dev data predicted label
print(metrics_calculator(y_pred_dev, y_dev0[:-2]))

(0.7864010120177103, 0.7852822580645161, 0.7858412368659745, 0.7852822580645161, 0.7852822580645161, 0.7852822580645161)


In [ ]:
# saving the trained model
model.save('CNN_XLNet.h5')  # creates a HDF5 file 'CNN_XLNet.h5'